In [1]:
from SmileGAN.Smile_GAN_clustering import single_model_clustering, cross_validated_clustering, clustering_result
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

C:\Users\drizzle\anaconda3\envs\deepcare\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d1_2 = pd.read_csv('../tadpole/TADPOLE_D1_D2.csv')

C:\Users\drizzle\AppData\Local\Temp\ipykernel_224\4265215034.py:1: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,72

In [3]:
dx_id_dict = {}
for i in d1_2['DX_bl'].unique():
    if i == 'CN':
        dx_id_dict[i] = 1
    else:
        dx_id_dict[i] = -1

In [4]:
roi_list = sorted(list(filter(lambda x: x.startswith('ST'),list(d1_2.columns))))[:-5]

In [5]:
roi_type = list(set([x.split('_')[0][-2:] for x in roi_list]))

In [6]:
print(roi_type)

['SV', 'CV', 'SA', 'TA', 'TS']


In [7]:
TA_list = [x for x in roi_list if x.split('_')[0][-2:] == 'TA']
SA_list = [x for x in roi_list if x.split('_')[0][-2:] == 'SA']
TS_list = [x for x in roi_list if x.split('_')[0][-2:] == 'TS']
SV_list = [x for x in roi_list if x.split('_')[0][-2:] == 'SV']
CV_list = [x for x in roi_list if x.split('_')[0][-2:] == 'CV']

In [8]:
def get_type_df(df,roi_list,dx_id_dict):
    clms = ['RID','DX_bl','AGE']
    clms += roi_list
    data = df[clms].rename(columns={"RID": "participant_id", "DX_bl": "diagnosis"})
    data = data.replace(' ',np.nan)
    data['diagnosis'] = data['diagnosis'].map(dx_id_dict)
    data.dropna(axis=1,how='all',inplace=True)
    data = data.dropna(axis=1,thresh = 7650).apply(pd.to_numeric)
    data = data.groupby('participant_id').first().reset_index()
    
    # Change to ROI0 - ROIn
    clm_name = {}
    clm_names = data.columns[3:]
    for i in range(len(clm_names)):
        clm_name[clm_names[i]] = f'ROI{i}'
    data = data.rename(columns=clm_name)
    train_id,test_id = train_test_split(data['participant_id'],test_size=0.2,random_state=42)
    train = data[data['participant_id'].isin(train_id)]
    train = train.apply(lambda x: x.fillna(x.mean()),axis=0)
    test = data[data['participant_id'].isin(test_id)]
    test = test.apply(lambda x: x.fillna(x.mean()),axis=0)
    
    return train, test

In [9]:
TA_train,TA_test = get_type_df(d1_2, TA_list, dx_id_dict)

In [10]:
len(TA_train),len(TA_train['participant_id'].unique())

(1389, 1389)

In [11]:
TA_train.shape

(1389, 71)

In [12]:
TA_train

,participant_id,diagnosis,AGE,ROI0,ROI1,ROI2,ROI3,ROI4,ROI5,ROI6,...,ROI58,ROI59,ROI60,ROI61,ROI62,ROI63,ROI64,ROI65,ROI66,ROI67
0,2,1,74.3,2.332,3.190,2.462,2.301,2.141,1.529,1.766,...,2.388,2.727,2.243,2.901,2.429,2.085,2.344,2.079,2.077,3.028
1,3,-1,81.3,2.121,2.214,2.295,2.280,2.323,1.294,1.675,...,2.603,2.318,2.190,2.377,2.052,1.994,2.173,1.765,1.942,2.545
2,4,-1,67.5,1.911,2.684,2.332,2.523,1.999,1.390,1.934,...,2.286,2.550,2.202,2.756,2.495,1.890,2.428,1.929,2.180,2.670
3,5,1,73.7,2.065,2.650,2.578,2.446,2.099,1.373,1.898,...,2.783,2.626,2.306,2.761,2.451,1.976,2.571,1.914,2.543,2.911
4,6,-1,80.4,2.301,2.212,2.480,2.870,2.264,1.305,1.724,...,2.780,2.398,2.419,2.768,2.745,2.257,2.456,1.790,2.141,2.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,5289,-1,59.7,2.359,3.170,2.408,2.562,2.260,1.583,1.898,...,2.426,2.677,2.415,2.987,2.612,2.355,2.375,2.081,2.195,2.965
1732,5290,-1,67.0,2.318,2.922,2.584,2.508,2.510,1.964,2.151,...,2.477,2.777,2.465,2.725,2.445,2.279,2.561,2.117,2.204,2.933
1733,5292,-1,74.3,2.347,2.868,2.550,2.440,2.343,1.500,2.031,...,2.742,2.819,2.406,2.701,2.337,2.090,2.403,2.135,2.115,2.819
1734,5294,-1,68.5,2.315,2.966,2.487,2.646,2.459,1.822,2.049,...,2.564,2.665,2.546,2.750,2.353,2.266,2.598,2.146,2.347,2.888


In [13]:
output_dir = "PATH_OUTPUT_DIR"
ncluster = 3
start_saving_epoch = 200
max_epoch = 500
## three parameters for stopping threshold
WD = 0.10
AQ = 20
cluster_loss = 0.0015
## one parameter for consensus method
consensus_type = "highest_matching_clustering"

fold_number = 10  # number of folds the leave-out cv runs
data_fraction = 0.8 # fraction of data used in each fold
cross_validated_clustering(TA_train, ncluster, fold_number, data_fraction, start_saving_epoch, max_epoch, output_dir, WD, AQ, cluster_loss, consensus_type)

Start Smile-GAN for semi-supervised clustering
****** Starting training of Fold 0 ******


 10%|█         | 50/500 [00:02<00:20, 22.02it/s]


****** Model not converging or not converged at max interation, Start retraining ******


 10%|█         | 50/500 [00:02<00:19, 22.96it/s]


****** Model not converging or not converged at max interation, Start retraining ******


 10%|█         | 50/500 [00:02<00:19, 22.92it/s]


****** Model not converging or not converged at max interation, Start retraining ******


  3%|▎         | 16/500 [00:00<00:21, 22.42it/s]
KeyboardInterrupt



In [14]:
output_dir = "PATH_OUTPUT_DIR"
ncluster = 3
start_saving_epoch = 200
max_epoch = 500
## three parameters for stopping threshold
WD = 0.10
AQ = 20
cluster_loss = 0.0015
## one parameter for consensus method
consensus_type = "highest_matching_clustering"

fold_number = 10  # number of folds the leave-out cv runs
data_fraction = 0.8 # fraction of data used in each fold
cross_validated_clustering(TA_test, ncluster, fold_number, data_fraction, start_saving_epoch, max_epoch, output_dir, WD, AQ, cluster_loss, consensus_type)

Start Smile-GAN for semi-supervised clustering
****** Starting training of Fold 0 ******



 10%|█         | 50/500 [00:00<00:04, 110.61it/s]


****** Model not converging or not converged at max interation, Start retraining ******



 10%|█         | 50/500 [00:00<00:03, 116.77it/s]


****** Model not converging or not converged at max interation, Start retraining ******


...

****** Model not converging or not converged at max interation, Start retraining ******



 10%|█         | 50/500 [00:00<00:04, 111.69it/s]


****** Model not converging or not converged at max interation, Start retraining ******



  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
output_dir = "../output/"
ncluster = 3
start_saving_epoch = 9000
max_epoch = 14000

In [19]:
## three parameters for stopping threshold
# WD = 0.10
# AQ = 20
# cluster_loss = 0.0015
# batch_size = 500
# WD = 1
# AQ = 20
# cluster_loss = 5
WD = 0.1
AQ = 70
cluster_loss = 0.002
batch_size = 70
## one parameter for consensus method
consensus_type = "highest_matching_clustering"

In [17]:
fold_number = 10  # number of folds the leave-out cv runs
data_fraction = 0.8 # fraction of data used in each fold
cross_validated_clustering(TA_train, ncluster, fold_number, data_fraction, start_saving_epoch, max_epoch,
                           output_dir, WD, AQ, cluster_loss, consensus_type)

Start Smile-GAN for semi-supervised clustering
****** Starting training of Fold 0 ******


 10%|█         | 50/500 [00:02<00:20, 22.36it/s]


****** Model not converging or not converged at max interation, Start retraining ******


  7%|▋         | 35/500 [00:01<00:21, 22.05it/s]
KeyboardInterrupt

  7%|▋         | 37/500 [00:17<00:20, 22.05it/s]